для монтирования яндекс диска

In [ ]:
!apt-get -q install davfs2

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libneon27
The following NEW packages will be installed:
  davfs2 libneon27
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 258 kB of archives.
After this operation, 627 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libneon27 amd64 0.32.2-1 [102 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 davfs2 amd64 1.6.1-1 [155 kB]
Fetched 258 kB in 0s (1,048 kB/s)
Preconfiguring packages ...
Selecting previously unselected package libneon27:amd64.
(Reading database ... 120899 files and directories currently installed.)
Preparing to unpack .../libneon27_0.32.2-1_amd64.deb ...
Unpacking libneon27:amd64 (0.32.2-1) ...
Selecting previously unselected package davfs2.
Preparing to unpack .../davfs2_1.6.1-1_amd64.deb ...
Unpacking davfs2 (1.6.1-1) ...
Setting up libneon

In [ ]:
!mkdir /content/yandexdisk
!mount -t davfs https://webdav.yandex.ru /content/yandexdisk

Please enter the username to authenticate with server
https://webdav.yandex.ru or hit enter for none.
  Username: paul.khimyack
Please enter the password to authenticate user paul.khimyack with server
https://webdav.yandex.ru or hit enter for none.
  Password:  


In [ ]:
!pip -q install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 43.1 MB/s eta 0:00:00


In [ ]:
import cv2
import os
import tqdm

import glob

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.framework.formats import landmark_pb2
from mediapipe import solutions
import pandas as pd


In [ ]:
CFG_DATASET_DIR = "/content/yandexdisk/uii/internship/KARATE_2step_s/dataset/"
CFG_DATASET_VIDEOS = "/content/yandexdisk/uii/internship/KARATE_2step_s/dataset/videos/"
dataset_directory = CFG_DATASET_DIR
video_directory = CFG_DATASET_VIDEOS
z_fill_num = 4 # для паддинга вида 0001

In [ ]:
#def list_files(startpath):
#    file_list = []
#    for root, dirs, files in os.walk(startpath):
#        for file in files:
#            file_list.append(os.path.join(root, file))
#    return file_list
#
#file_list = list_files(video_directory)

## Выводим список файлов с относительными путями
#relative_file_list = [os.path.relpath(file, video_directory) for file in file_list]


In [ ]:
## Создаем DataFrame
#df = pd.DataFrame({"file_name": relative_file_list})
#df.insert(0, 'file_id', range(0, len(df)))
## Выводим DataFrame
#df.to_excel(
#    os.path.join(dataset_directory, "files_indices.xlsx")
#)
df = pd.read_excel(
     os.path.join(dataset_directory, "files_indices.xlsx"),
     index_col = 0
)
display(df)

,file_id,file_name
0,0,hits_katas_3cams/гьяку дзуки/гьяку дзуки права...
1,1,hits_katas_3cams/гьяку дзуки/гьяку дзуки права...
2,2,hits_katas_3cams/гьяку дзуки/гьяку дзуки правы...
3,3,hits_katas_3cams/гьяку дзуки/центральная камер...
4,4,hits_katas_3cams/гьяку дзуки другое расположен...
...,...,...
309,309,single_hit/test/УРАКЕН УЧИ/ПРАВИЛЬНО/20230814_...
310,310,single_hit/test/УЧИ УКЕ/ОШИБКА/перекрут корпус...
311,311,single_hit/test/УЧИ УКЕ/ОШИБКА/перекрут корпус...
312,312,single_hit/test/УЧИ УКЕ/ПРАВИЛЬНО/20230814_170...


In [ ]:
df = df.loc[df['file_name'].str.lower().str.contains('.mp4')].copy()

In [ ]:
df.to_excel(os.path.join(CFG_DATASET_DIR, "files_indices.xlsx"))

Разметка медиапайп

In [ ]:
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow

from pandas.core.arrays.sparse.array import splib

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import json

In [ ]:
class Marker():
  def __init__(self, pose_tracker, save_dir, z_fill_num = 4):
    self.pose_tracker  = pose_tracker
    self.z_fill_num = z_fill_num
    self.save_dir = save_dir

  def _mark_file(self, file_name):
      video_cap = cv2.VideoCapture(file_name)
      # Get some video parameters to generate output video with classificaiton.
      video_n_frames = video_cap.get(cv2.CAP_PROP_FRAME_COUNT)
      video_fps = video_cap.get(cv2.CAP_PROP_FPS)
      video_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
      video_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


      frame_idx = 0
      till_frame_no = None
      output_frame = None

      result_out = {
          "frame_no":[],
          "landmarks": [],
      }

      with tqdm.tqdm(total=video_n_frames, position=0, leave=True) as pbar:
        while video_cap.isOpened():
          # Get next frame of the video.
          success, input_frame = video_cap.read()
          if not success:
            break
          if (till_frame_no is not None) and (frame_idx >= till_frame_no):
            break

          # Run pose tracker.
          input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)
          mp_image = mp.Image(image_format = mp.ImageFormat.SRGB, data=np.array(input_frame))
          detection_result = pose_tracker.detect(mp_image)
          # Draw pose prediction.
          output_frame = input_frame.copy()

          if (detection_result is not None) and len(detection_result.pose_landmarks) > 0 :
            assert hasattr(detection_result, 'pose_landmarks')  , "Unexpected pose_landmakrs attribute not present {}"
            #только с одной позой работаем
            pose_landmarks = detection_result.pose_landmarks[0]
            #for pose_landmarks in pose_landmarks_instances:
            # Get landmarks.
            #frame_height, frame_width = output_frame.shape[0], output_frame.shape[1]
            pose_landmarks =np.array([[lmk.x , lmk.y , lmk.z , lmk.visibility, lmk.presence] for lmk in pose_landmarks ])
            assert pose_landmarks.shape == (33, 5), 'Unexpected landmarks shape: {}'.format(pose_landmarks.shape)
            pose_landmarks = json.dumps(pose_landmarks.tolist())
            #embeddings = pose_embedder(landmarks=pose_landmarks)

          else:
            #If no landamks then use prev embeddings
            #embeddings = None
            pose_landmarks = np.nan
            print("landmark was not detected")

          result_out["landmarks"].append(pose_landmarks)
          result_out["frame_no"].append(frame_idx)
          frame_idx += 1
          pbar.update()

      return result_out

  def __call__(self, dataframe):
    for idx, row in dataframe.iterrows():

      video_file = os.path.join(video_directory, row['file_name'])
      video_file_id = row["file_id"]

      print(f"file_id:{video_file_id} - {video_file}")
      result_out = self._mark_file(video_file)

      df_result = pd.DataFrame(result_out)
      df_result["file_id"] = video_file_id

      padded_numeric_part = str(video_file_id).zfill(self.z_fill_num)

      xlsx_file = os.path.join(self.save_dir , f"frames_mediapipe_landmarks_file_id{padded_numeric_part}.xlsx")
      print(f"запись {xlsx_file}")
      df_result.to_excel(xlsx_file)



In [ ]:
pd_files_indices = pd.read_excel(os.path.join(CFG_DATASET_DIR, "files_indices.xlsx" ),index_col = 0)

In [ ]:
pd_files_indices

,file_id,file_name
0,0,hits_katas_3cams/гьяку дзуки/гьяку дзуки права...
1,1,hits_katas_3cams/гьяку дзуки/гьяку дзуки права...
2,2,hits_katas_3cams/гьяку дзуки/гьяку дзуки правы...
3,3,hits_katas_3cams/гьяку дзуки/центральная камер...
4,4,hits_katas_3cams/гьяку дзуки другое расположен...
...,...,...
309,309,single_hit/test/УРАКЕН УЧИ/ПРАВИЛЬНО/20230814_...
310,310,single_hit/test/УЧИ УКЕ/ОШИБКА/перекрут корпус...
311,311,single_hit/test/УЧИ УКЕ/ОШИБКА/перекрут корпус...
312,312,single_hit/test/УЧИ УКЕ/ПРАВИЛЬНО/20230814_170...


Продолжаю размечать датасет

In [ ]:
import enum
class Hits(enum.Enum):
  UNKNOWN_HIT = 0
  GEDAN_BARAY = 1
  GYAKU_DZUKY = 2
  KIDZAMI_DSUKI = 3
  MAVASHI_GERI = 4
  MAVASHI_DZUKI = 5
  MAY_GERI_KEAGE = 6
  SOTO_UKE = 7
  URAKEN_UCHI = 8
  URA_MAVASHI_GERI = 9
  UCHI_UKE = 10

hits = [
        (Hits.UNKNOWN_HIT.value,      "UNKNOWN",          ""),
        (Hits.GEDAN_BARAY.value,      'ГЕДАН БАРАЙ',     "БЛОК ПЕРЕДНЕЙ РУКОЙ"),
        (Hits.GYAKU_DZUKY.value,      'ГЬЯКУ ДЗУКИ',     "ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ"),
        (Hits.KIDZAMI_DSUKI.value,    'КИДЗАМИ ДЗУКИ',   "ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ"),
        (Hits.MAVASHI_GERI.value,     'МАВАШИ ГЕРИ',     "КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ"),
        (Hits.MAVASHI_DZUKI.value,    "МАВАШИ  ДЗУКИ",   "КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ"),
        (Hits.MAY_GERI_KEAGE.value,   'МАЙ ГЕРИ КЕАГЕ',  'ПРЯМОЙ УДАР ЗАДНЕЙ НОГОЙ'),
        (Hits.SOTO_UKE.value,         'СОТО УКЕ',        'БЛОК ПЕРЕДНЕЙ РУКОЙ'),
        (Hits.URAKEN_UCHI.value,      'УРАКЕН УЧИ',      'КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ'),
        (Hits.URA_MAVASHI_GERI.value, 'УРА МАВАШИ ГЕРИ', 'ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ'),
        (Hits.UCHI_UKE.value,         'УЧИ УКЕ',         'БЛОК ПЕРЕДНЕЙ РУКОЙ')
      ]

pd_hits = pd.DataFrame(hits, columns=['hit_id', 'hit_name', 'hit_description'])
pd_hits.to_excel(os.path.join(CFG_DATASET_DIR, "hits.xlsx"))

NameError: ignored

In [ ]:
pd_single_hit = pd_files_indices[pd_files_indices["file_name"].str.contains("single_hit")].copy()


In [ ]:

pd_single_hit['correct'] = pd.NA

pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("correct"), "correct"] = True
pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("правиль"), "correct"] = True

pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("error"), "correct"] = False
pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("ошибка"), "correct"] = False

pd_single_hit['train'] =True
pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("test"), "train"] = False




In [ ]:
hits_hash = {hit_tupple[0]:hit_tupple[1] for hit_tupple in hits}

In [ ]:
pd_single_hit["hit_id"] = pd.NA
for hit_key in  hits_hash.keys():
  pd_single_hit.loc[pd_single_hit["file_name"].str.contains(hits_hash[hit_key]), 'hit_id'] = hit_key

In [ ]:
pd_single_hit["hit_name"] = pd.NA
for hit_key in  hits_hash.keys():
  pd_single_hit.loc[pd_single_hit["file_name"].str.contains(hits_hash[hit_key]), 'hit_name'] = hits_hash[hit_key]

In [ ]:
pd_single_hit.loc[:, "camera"] = pd.NA
pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("бок"), 'camera'] = "side"
pd_single_hit.loc[pd_single_hit["file_name"].str.lower().str.contains("фронт"), 'camera'] = "front"

In [ ]:
pd_single_hit.to_excel(os.path.join(CFG_DATASET_DIR, "single_hit.xlsx"))

In [ ]:
#oksana_all_df = pd.read_csv("/content/drive/MyDrive/uii/internships/intern_karate/dataset_all.csv")

In [ ]:
pd_single_hit = pd.read_excel(os.path.join(CFG_DATASET_DIR, "single_hit.xlsx"), index_col=0)



In [ ]:
#pd_single_hit.to_excel(os.path.join(CFG_DATASET_DIR, "single_hit.xlsx"))


In [ ]:
#add videos metadata to dataframes
COLUMN_VIDEO_FPS = "video_fps"
COLUMN_VIDEO_WIDTH = "video_width"
COLUMN_VIDEO_HEIGHT = "video_height"
COLUMN_VIDEO_FRAMES_COUNT = "video_frames_count"





Метаданные видео

In [ ]:
for idx, row in pd_single_hit[pd_single_hit['video_fps'] == 0].iterrows():
  print("current file.." , row["file_name"])
  video_cap = cv2.VideoCapture(os.path.join(CFG_DATASET_VIDEOS, row['file_name']))
  # Get some video parameters to generate output video with classificaiton.

  video_n_frames = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
  video_fps = int(video_cap.get(cv2.CAP_PROP_FPS))
  video_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  video_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  pd_single_hit.at[idx, COLUMN_VIDEO_FPS] = video_fps
  pd_single_hit.at[idx, COLUMN_VIDEO_WIDTH] = video_width
  pd_single_hit.at[idx, COLUMN_VIDEO_HEIGHT] = video_height
  pd_single_hit.at[idx, COLUMN_VIDEO_FRAMES_COUNT] = video_n_frames


  video_cap.release()


pd_single_hit[COLUMN_VIDEO_FPS] = pd_single_hit[COLUMN_VIDEO_FPS].astype(int)
pd_single_hit[COLUMN_VIDEO_WIDTH] = pd_single_hit[COLUMN_VIDEO_WIDTH].astype(int)
pd_single_hit[COLUMN_VIDEO_HEIGHT] = pd_single_hit[COLUMN_VIDEO_HEIGHT].astype(int)
pd_single_hit[COLUMN_VIDEO_FRAMES_COUNT] = pd_single_hit[COLUMN_VIDEO_FRAMES_COUNT].astype(int)

In [ ]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task


In [ ]:
# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        num_poses = 1,
        min_pose_detection_confidence = 0.5,
        min_pose_presence_confidence = 0.5,
        min_tracking_confidence = 0.5,
        output_segmentation_masks = False
    )

with vision.PoseLandmarker.create_from_options(options) as pose_tracker:
  marker_executor = Marker(pose_tracker, save_dir = CFG_DATASET_DIR,z_fill_num= z_fill_num)
  marker_executor(
      #pd_single_hit[pd_single_hit['hit_id'] == Hits.MAVASHI_DZUKI.value ]
      pd_single_hit[pd_single_hit['file_id'].isin([218, 260, 269])]
  )


file_id:218 - /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/videos/single_hit/error/МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)/С БОКУ/Максим МАВАШИ ГЕРИ (без хики аши 2).mp4


100%|██████████| 827/827.0 [03:12<00:00,  4.29it/s]


запись /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/frames_mediapipe_landmarks_file_id0218.xlsx
file_id:260 - /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/videos/single_hit/error/УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)/ФРОНТАЛЬНО/Максим УРА МАВАШИ ГЕРИ (прямая нога 1).mp4


 10%|█         | 82/792.0 [01:17<06:50,  1.73it/s]

landmark was not detected


...

landmark was not detected


100%|██████████| 792/792.0 [04:02<00:00,  3.27it/s]


запись /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/frames_mediapipe_landmarks_file_id0260.xlsx
file_id:269 - /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/videos/single_hit/error/УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)/ФРОНТАЛЬНО/Андрей УРАКЕН УЧИ (положение кулака 1).mp4


100%|██████████| 545/545.0 [03:08<00:00,  2.90it/s]


запись /content/yandexdisk/uii/internship/KARATE_2step_s/dataset/frames_mediapipe_landmarks_file_id0269.xlsx


In [21]:

files = glob.glob(os.path.join(dataset_directory, "frames_mediapipe_landmarks_file_id*.xlsx"))
files = sorted(files)
dfs = []
for file_name in files:
  dfs.append(pd.read_excel(file_name, index_col = 0))
merged_vertically_df = pd.concat(dfs, ignore_index=True)
merged_vertically_df.to_excel(
    os.path.join(dataset_directory, "frames_mediapipe_landmarks_single_hits.xlsx")
)


In [ ]:
merged_vertically_df

In [ ]:
merged_vertically_df.to_excel(
    os.path.join(dataset_directory, "frames_mediapipe_landmarks_single_hits.xlsx")
)
